Won't currently work. To be updated. Skip for now

In [1]:
import pytest

In [5]:
!pytest ../mne_features/tests/test_braindecode_features.py -vvv

Traceback (most recent call last):
  File "/home/gemeinl/anaconda3/bin/pytest", line 7, in <module>
    from py.test import main
ModuleNotFoundError: No module named 'py'


In [1]:
import sys

from sklearn.ensemble import RandomForestClassifier

from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.classification import AutoSklearnClassifier

sys.path.insert(0, '/home/gemeinl/code/braindecode-features/')
from braindecode_features import *

/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
subject_id = 1
train_or_test = 'train'
path = f'/home/gemeinl/code/mne-features/examples/{subject_id}/{train_or_test}/'
n_jobs = 2
seed = 20210408
np.random.seed(seed)
only_last_fold = True
# set to integer to enable autosklearn usage with specified integer as training time in minutes
n_min = None  # None/2/20 
drop_a_window = 0

clfs = []
rf = RandomForestClassifier(
    n_estimators=750, 
    random_state=seed
)
clfs.append(rf)
if n_min is not None:
    asc = AutoSklearnClassifier( 
        time_left_for_this_task=60*n_min,
        ml_memory_limit=8192,
        n_jobs=n_jobs,
        seed=seed,
        initial_configurations_via_metalearning=0,
    )
    clfs.append(asc)

df = read_features(
    path=path,
    n_jobs=n_jobs,
)
if drop_a_window is not None:
    df = drop_window(df, drop_a_window)
# try different feature usages
agg_funcs = [None, None, np.mean]
windows_as_examples_ = [True, False, False]
for clf in clfs:
    # TODO: add filter_df of feature dataframe
    for agg_func, windows_as_examples in zip(agg_funcs, windows_as_examples_):
        cross_validate(
            df=df,
            clf=clf,
            subject_id=subject_id,
            only_last_fold=only_last_fold,
            agg_func=agg_func,
            windows_as_examples=windows_as_examples,
            out_path=None,
        )

/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                estimator agg_func  windows_as_examples  window_accuracy  \
0  RandomForestClassifier     None                 True         0.753125   

   trial_accuracy  
0        0.859375  


/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


                estimator agg_func  windows_as_examples  window_accuracy  \
0  RandomForestClassifier     None                False         0.921875   

   trial_accuracy  
0        0.921875  
                estimator agg_func  windows_as_examples  window_accuracy  \
0  RandomForestClassifier     mean                False          0.90625   

   trial_accuracy  
0         0.90625  


Dropping a window is not yet properly working... trying to fix below

In [2]:
import sys
sys.path.insert(0, '/home/gemeinl/code/mne-features/')
from mne_features.braindecode_features import read_features, filter_df, prepare_features, drop_window
subject_id = 1
train_or_test = 'train'
path = f'/home/gemeinl/code/mne-features/examples/{subject_id}/{train_or_test}/'
n_jobs = 2

In [3]:
df = read_features(
    path=path,
    n_jobs=n_jobs,
)

In [33]:
df = orig_df.copy()

In [34]:
window_i = 0

In [35]:
df = df[df.Window != window_i]
df.loc[:, 'Window'] = df.loc[:, 'Window'].subtract(df.Window > window_i)

In [36]:
df

Domain    Trial Window Target       Time                                \
Feature                       covariance                                 
Channel                            C1-C2     C1-C3     C1-C4     C1-C5   
Frequency                          32-99     32-99     32-99     32-99   
1             0      0      2  -0.001870  1.113724  0.136482  1.203941   
2             0      1      2   0.070855  1.007939  0.231097  1.098321   
3             0      2      2  -0.066271  0.662896 -0.045463  0.774153   
4             0      3      2   0.021986  0.649025  0.095147  0.757287   
5             0      4      2   0.055081  0.815854  0.140423  0.934071   
...         ...    ...    ...        ...       ...       ...       ...   
1915        319      0      1   0.077357  0.835075  0.204692  1.110051   
1916        319      1      1   0.014710  1.053171  0.133093  1.443284   
1917        319      2      1  -0.014155  0.930813  0.126767  1.221783   
1918        319      3      1  -0.072016  0.764413 -0.043669  0.823206   
1919        319      4      1  -0.027847  0.745496 -0.011004  0.883799   

Domain                                   ...        Connectivity              \
Feature                                  ... phase_locking_value               
Channel       C1-C6  C1-CCP1h  C1-CCP2h  ...         FFC2h-FFC3h FFC2h-FFC4h   
Frequency     32-99     32-99     32-99  ...             114-123     114-123   
1          0.407750  0.298852  0.021070  ...            0.129199    0.429823   
2          0.477215  0.320023  0.103361  ...            0.153915    0.454811   
3          0.115176  0.232299  0.036610  ...            0.208477    0.419674   
4          0.284628  0.224690  0.046021  ...            0.169760    0.499325   
5          0.356129  0.255061  0.006065  ...            0.208279    0.557658   
...             ...       ...       ...  ...                 ...         ...   
1915       0.328407  0.255151  0.075200  ...            0.140481    0.319087   
1916       0.354250  0.311021  0.067232  ...            0.009019    0.380174   
1917       0.263251  0.262333  0.063229  ...            0.093721    0.397786   
1918       0.020542  0.232337 -0.008095  ...            0.067137    0.439482   
1919       0.154272  0.237186  0.006813  ...            0.072494    0.546638   

Domain                                                                 \
Feature                                                                 
Channel   FFC2h-FFC5h FFC2h-FFC6h FFC3h-FFC4h FFC3h-FFC5h FFC3h-FFC6h   
Frequency     114-123     114-123     114-123     114-123     114-123   
1            0.132399    0.437792    0.195142    0.515190    0.126852   
2            0.026807    0.438907    0.188200    0.184972    0.092388   
3            0.136822    0.320057    0.017053    0.235491    0.065181   
4            0.068676    0.443327    0.163167    0.546267    0.136396   
5            0.134371    0.560786    0.168510    0.433024    0.203479   
...               ...         ...         ...         ...         ...   
1915         0.206981    0.174634    0.167401    0.350229    0.088363   
1916         0.274268    0.275382    0.270115    0.392472    0.259021   
1917         0.342894    0.487223    0.141408    0.416177    0.137721   
1918         0.220237    0.455648    0.185133    0.492002    0.071936   
1919         0.125909    0.348256    0.026192    0.305117    0.287568   

Domain                                         
Feature                                        
Channel   FFC4h-FFC5h FFC4h-FFC6h FFC5h-FFC6h  
Frequency     114-123     114-123     114-123  
1            0.210248    0.678703    0.219449  
2            0.096713    0.790124    0.153444  
3            0.176999    0.724462    0.057672  
4            0.101349    0.736350    0.112429  
5            0.109671    0.827177    0.100483  
...               ...         ...         ...  
1915         0.343059    0.567041    0.195183  
1916         0.184441    0.809096    0.203944  
1917         0.1

In [2]:
cross_validate_main()

/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/gemeinl/anaconda3/envs/auto-sklearn2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
results = pd.read_csv(
    '/home/gemeinl/code/mne-features/examples/1/cv_results.csv',
    index_col=0
).reset_index(drop=True)

In [4]:
results.columns

Index(['subject', 'fold', 'estimator', 'window_accuracy', 'trial_accuracy',
       'predictions', 'targets', 'model', 'feature_names',
       'windows_as_examples', 'agg_func'],
      dtype='object')

In [6]:
results[['subject', 'estimator', 'window_accuracy', 'trial_accuracy', 'agg_func', 'windows_as_examples']]

,subject,estimator,window_accuracy,trial_accuracy,agg_func,windows_as_examples
0,1,RandomForestClassifier,0.753125,0.859375,NaN,True
1,1,RandomForestClassifier,0.921875,0.921875,NaN,False
2,1,RandomForestClassifier,0.906250,0.906250,mean,False
